In [33]:
#bibliotecas
import numpy as np
import random

In [34]:
#inicializar ambiente, acoes, estados, recompensas, transicoes
#   estados
# 12 13 14 15
# 8  9  10 11
# 4  5  6  7
# 0  1  2  3

# se state == 0 || 4 || 8 || 12 -> nao posso mover para esquerda
# se state == 3 || 7 || 11 || 14 -> nao posso mover para direita
# se state == 0 || 1 || 2 || 3 -> nao posso mover para baixo
# se state == 12 || 13 || 14 || 15 -> nao posso mover para cima

estados_limite_cima = [12, 13, 14, 15]
estados_limite_baixo = [0, 1, 2, 3]
estados_limite_direita = [3, 7, 11, 14]
estados_limite_esquerda = [0, 4, 8, 12]

# V V V P
# W O P V
# V V V V
# I V P V

# V -> espaço vazio
# W -> wumpus
# P -> poço
# I -> posicao inicial do guerreiro


estado_ouro = 9
estado_wumpus = 8
estados_pocos = [2, 10, 15]

#   estado final
#   pode ser o ouro, wumpus vivo ou poços

estados_finais = [2, 8, 9, 10, 15]
wumpus_vivo = True


In [35]:
# inicializa o guerreiro, suas acoes e recompensas

# N = 1
# O = 2
# L = 3
# S = 4

global face_guerreiro
face_guerreiro = 3 # o guerreiro inicia olhando para a direita
global estado_guerreiro
estado_guerreiro = 0

# acoes
CIMA = 1
BAIXO = 2
DIREITA = 3
ESQUERDA = 4
ATIRAR = 5
PEGAR = 6
ACOES = [CIMA, BAIXO, DIREITA, ESQUERDA, ATIRAR, PEGAR]

#   recompensas
recompensa_estado_vazio = -1
recompensa_poco = -100
recompensa_wumpus = -100
recompensa_matar_wumpus = 50
recompensa_pegar_ouro = 500



In [41]:
#funcao basicas

def acao_permitida(acao):
    if acao == CIMA and estado_guerreiro in estados_limite_cima:
        return False
    elif acao == BAIXO and estado_guerreiro in estados_limite_baixo:
        return False
    elif acao == DIREITA and estado_guerreiro in estados_limite_direita:
        return False
    elif acao == ESQUERDA and estado_guerreiro in estados_limite_esquerda:
        return False
    else:
        return True

def novo_estado_guerreiro(acao):
    if acao == CIMA:
        face_guerreiro = 1
        return (estado_guerreiro+4)
    elif acao == BAIXO:
        face_guerreiro = 4
        return (estado_guerreiro-4)
    elif acao == DIREITA:
        face_guerreiro = 3
        return (estado_guerreiro+1)
    elif acao == ESQUERDA:
        face_guerreiro = 2
        return (estado_guerreiro-1)
    else: # guerreiro atirou ou 'pegou' o ouro
        return estado_guerreiro
  
def flecha_mata_wumpus():
    if not wumpus_vivo:
        return False
    estado_flecha = estado_guerreiro
    if face_guerreiro == 1:
        while estado_flecha not in estados_limite_cima:
            if(estado_flecha == estado_wumpus):
                wumpus_vivo = False
                return True
            estado_flecha += 4
        return False
    elif face_guerreiro == 2:
        while estado_flecha not in estados_limite_esquerda:
            if(estado_flecha == estado_wumpus):
                wumpus_vivo = False
                return True
            estado_flecha -= 1
        return False
    elif face_guerreiro == 3:
        while estado_flecha not in estados_limite_direita:
            if(estado_flecha == estado_wumpus):
                wumpus_vivo = False
                return True
            estado_flecha += 1
        return False
    else:
        while estado_flecha not in estados_limite_baixo:
            if(estado_flecha == estado_wumpus):
                wumpus_vivo = False
                return True
            estado_flecha -= 4
        return False
                
def recompensas(acao):
    if estado_guerreiro in estados_pocos:
        return recompensa_poco
    elif estado_guerreiro == estado_wumpus and wumpus_vivo:
        return recompensa_wumpus
    elif estado_guerreiro == estado_ouro and acao == PEGAR:
        return recompensa_pegar_ouro
    elif acao == ATIRAR:
        if flecha_mata_wumpus():
            return recompensa_matar_wumpus
        else:
            return recompensa_estado_vazio
    else:
        return recompensa_estado_vazio

def tomar_acao(estado_guerreiro):
    # exploration
    acao = random.choice(ACOES)
    print("Acao: ", acao)
    print("Estado inicial: ", estado_guerreiro)
    if acao_permitida(acao):
        estado_guerreiro = novo_estado_guerreiro(acao)

    recompensa = recompensas(acao)
    print("Novo estado: ", estado_guerreiro)
    print(recompensa)



In [43]:
tomar_acao(estado_guerreiro)
print(face_guerreiro)

Acao:  5
Estado inicial:  0


UnboundLocalError: local variable 'wumpus_vivo' referenced before assignment

In [ ]:
#q-learning
#q_table
#Cada linha representa os estados e cada coluna as ações possíveis em cada estado
#array([
#00      [0, 0, 0, 0, 0, 0]
#        ...,
#15      [0, 0, 0, 0, 0, 0]])


q_table = np.zeros((16, 4))
q_table

In [ ]:
# funcao para realizar uma acao


In [ ]:
#Treinamento: para cada estado, selecionar uma ação a partir deste estado
# Hyperparameters
alpha = 0.1 #taxa de aprendizado
gamma = 0.6 #fator de desconto
epsilon = 0.1 #parametro que auxilia a decidir entre exploration e explotation 
              #e evitar overfitting (escolha da mesma rota sempre)